<a href="https://colab.research.google.com/github/philippe753/CNN_project/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import glob
from matplotlib import pyplot as plt
import gc
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, AveragePooling2D, Conv2D, MaxPooling2D
from keras.preprocessing import image
from keras.optimizers import SGD, Adam
from sklearn.preprocessing import normalize
import cv2

In [ ]:
# unzip the data frame containing the celebraties images
!unzip "/content/drive/MyDrive/AI_dataset/img_align_celeba.zip"

In [2]:
# Importing data
data_Y = pd.read_excel("/content/drive/MyDrive/AI_dataset/list_attr_celeba.xlsx")
data_Y_columns = data_Y.iloc[0] # This contains the name of the features/columns of the dataframe.

In [3]:
# Inspecting Data

# plot of the first 5 columns of the raw data
print("Raw data frame: \n")
print(data_Y.head(5))


data_Y = data_Y.replace(-1, 0) # Replace -1 for a 0.
# Deleating the first row, since it contains it just a interger refereing to the total number of imgs.
first_row = data_Y.iloc[0] #grab the first row for the header
data_Y = data_Y[1:] #take the data less the header row
data_Y.columns = first_row #set the header row as the df header
print('New data frame: \n')
data_Y.head(5)

Raw data frame: 

       202599        Unnamed: 1  ...      Unnamed: 39 Unnamed: 40
0           #  5_o_Clock_Shadow  ...  Wearing_Necktie       Young
1  000001.jpg                -1  ...               -1           1
2  000002.jpg                -1  ...               -1           1
3  000003.jpg                -1  ...               -1           1
4  000004.jpg                -1  ...               -1           1

[5 rows x 41 columns]
New data frame: 



,#,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,Blurry,Brown_Hair,Bushy_Eyebrows,Chubby,Double_Chin,Eyeglasses,Goatee,Gray_Hair,Heavy_Makeup,High_Cheekbones,Male,Mouth_Slightly_Open,Mustache,Narrow_Eyes,No_Beard,Oval_Face,Pale_Skin,Pointy_Nose,Receding_Hairline,Rosy_Cheeks,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
1,000001.jpg,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1
2,000002.jpg,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,000003.jpg,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1
4,000004.jpg,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,1,0,1
5,000005.jpg,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1


## Description of our data Frame
As we can see bellow, most examples are not bold, do not have Clock shadow, do not have bangs, do not have blond hair, do not have double chin, do not have eyeglasses, do not have a Goatee, nor gray_hair, nor mustache, nor male skin, nor have a recending hairline, nor have rosy cheeks,nor side burns, nor wearing a hat, nor wearing a necktie.

The first model we'll learn is a  simple logisitc neural network to try to predict attrative and non attrative people. 

In [ ]:
data_Y.describe()

,#,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,Blurry,Brown_Hair,Bushy_Eyebrows,Chubby,Double_Chin,Eyeglasses,Goatee,Gray_Hair,Heavy_Makeup,High_Cheekbones,Male,Mouth_Slightly_Open,Mustache,Narrow_Eyes,No_Beard,Oval_Face,Pale_Skin,Pointy_Nose,Receding_Hairline,Rosy_Cheeks,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
count,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599,202599
unique,202599,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,158328.jpg,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
freq,1,180083,148509,103833,161153,198052,171890,153814,155083,154127,172616,192287,161027,173796,190936,193140,189406,189883,194100,124209,110410,118165,104657,194182,179270,169158,145032,193898,146389,186436,189284,191150,104930,160377,137855,164323,192781,106884,177686,187867,156734


## Randomly pick m images to be the training set.
Every image is flattened from a 3 dimmensional array to a 1-dimensional array and then standarized.

In [4]:
m = 2000  # number of examples to train the models
num_features = 116412 # num of features for each image
train_X = np.zeros((num_features, m))
train_Y = pd.DataFrame(data=data_Y_columns, columns=data_Y_columns)
# random numbers to selecet the training examples from the data frame.
rand_num = np.random.choice(range(202599), m, replace=False) 
img_path = np.sort(glob.glob("img_align_celeba/*.*"))

for i in range(0, m):
    img = cv2.imread(img_path[rand_num[i]])
    # flat image into an array.
    img_v = img.reshape((img.shape[0] * img.shape[1] * img.shape[2], 1))
    train_X[:, i] = img_v[:, 0]
    # find the equivalent samples in data_Y.
    train_Y = pd.concat([train_Y, data_Y[data_Y['#'] == data_Y.iloc[rand_num[i]][0]]], ignore_index=True)

# Standardize data
train_X = train_X / 255.

# For now, we are only interested on the attribute attractive.
train_Y_attractive = train_Y['Attractive']
train_Y_attractive = train_Y_attractive.values.reshape((1, -1))

In [ ]:
## Visualizing the training set. 
# Note that since our models were randomly chosen from the data frame, then this
# sample should be a good representation of the Data Frame.
print("train_X shape is: " + str(train_X.shape))
print("train_Y shape is: " + str(train_Y.shape))
print("\nDescription of the features of the training set:")
train_Y.describe()

train_X shape is: (116412, 2000)
train_Y shape is: (2000, 41)

Description of the features of the training set:


,#,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,Blond_Hair,Blurry,Brown_Hair,Bushy_Eyebrows,Chubby,Double_Chin,Eyeglasses,Goatee,Gray_Hair,Heavy_Makeup,High_Cheekbones,Male,Mouth_Slightly_Open,Mustache,Narrow_Eyes,No_Beard,Oval_Face,Pale_Skin,Pointy_Nose,Receding_Hairline,Rosy_Cheeks,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
count,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000
unique,2000,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,151433.jpg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
freq,1,1786,1476,1009,1601,1948,1735,1510,1546,1522,1697,1888,1573,1710,1884,1898,1870,1865,1906,1230,1088,1142,1013,1905,1728,1670,1447,1921,1445,1831,1875,1884,1018,1605,1346,1627,1891,1083,1772,1828,1540


## Perceptron model to predict attractive and non-attractive people.

We will implement the most basic implementation of a neural network to see how good a single layer neural network can accurately predict if a person is attractive or not. 

In [ ]:
# Implementing a 2 layer neural network

class TwoLayerNN:

    def __init__(self, learning_rate=0.01, num_ite=1000):
        self.learning_rate = learning_rate
        self.num_ite = num_ite
        self.num_x = num_features
        self.num_hidden = 20
        self.num_y = 1 # number of output units.
        self.W1 = None
        self.b1 = None
        self.W2 = None
        self.b2 = None

    def initialize_parameters(self, num_x, num_hidden, num_y):
        self.W1 = np.random.randn(num_hidden, num_x) * 0.01
        self.b1 = np.zeros((num_hidden, 1))
        self.W2 = np.random.randn(num_y, num_hidden) * 0.01
        self.b2 = np.zeros((num_hidden, 1))
        parameters = {"W1": self.W1, "b1": self.b1, "W2": self.W2, "b2": self.b2}
        return parameters

    def fit(self, X, Y):
        grads = {}
        costs = []                              # to keep track of the cost
        # Initialize parameters
        parameters = self.initialize_parameters(self.num_x, self.num_hidden, self.num_y)
        for i in range(0, self.num_ite):
            # Forward propagation: with a relu and then sigmoid activation function
            A1, cache1 = self.linear_activation_forward(X, self.W1, self.b1, "relu")
            A2, cache2 = self.linear_activation_forward(A1, self.W2, self.b2, "sigmoid")

            # Cost function
            cost  = self.compute_cost(A2, Y)

            # Initializing backward propagation
            dA2 = - (np.divide(Y, A2) - np.divide(1 - Y, 1 - A2))
            dA1, dW2, db2 = self.linear_activation_backward(dA2, cache2, "sigmoid")
            A0, dW1, db1 = self.linear_activation_backward(dA1, cache1, "relu")
            grads['dW1'] = dW1
            grads['db1'] = db1
            grads['dW2'] = dW2
            grads['db2'] = db2

            # Update the paramters.
            parameters = self.update_parameters(parameters, grads)
            self.W1 = parameters["W1"]
            self.b1 = parameters["b1"]
            self.W2 = parameters["W2"]
            self.b2 = parameters["b2"]

    def linear_forward(self, A, W, b):
        # Implement the linear part of a layer's forward propagation.
        Z = np.dot(W, A) + b
        cache = (A, W, b)
        return Z, cache

    def linear_activation_forward(self, A_prev, W, b, activation):
        # Implement the forward propagation for the LINEAR->ACTIVATION layer
        Z, linear_cache = self.linear_forward(A_prev, W, b)
        if activation == "sigmoid":
            A = 1/(1 + np.exp(Z))
        elif activation == "relu":
            A = np.maximum(Z, 0)
        cache = (linear_cache, Z)  # note th
        return A, cache

    def linear_backward(self, dZ, cache):
        # linear backword propagation
        A_prev, W, b = cache
        m = A_prev.shape[1]

        dW = (1/m) * np.dot(dZ, A_prev.T)
        db = (1/m) * np.sum(dZ, axis = 1, keepdims=True)
        print('dz: ' + str(dZ.shape))
        print('W.T: ' + str(W.T.shape))
        dA_prev = np.dot(W.T, dZ)

    def linear_activation_backward(self, dA, cache, activation):
        # Backward propagation.
        linear_cache, activation_cache = cache
        Z = activation_cache
        if activation == "relu":
          # Implement Back propagation witha relu activation function
            dZ = self.relu_backward(dA, activation_cache)
            dA_prev, dW, db = self.linear_backward(dZ, linear_cache)
        elif activation == "sigmoid":
          # Implement Back propagation witha sigmoid activation function
            dsig = 1/(1+np.exp(-Z)) * (1 - 1/(1+np.exp(-Z)))
            dZ = dA * dsig    #= self.sigmoid_backward(dA, activation_cache)
            dA_prev, dW, db = self.linear_backward(dZ, linear_cache)
        return dA_prev, dW, db

    def compute_cost(self, AL, Y):
        cost = -(1/m) * (np.dot(Y, np.log(AL).T) + np.dot((1 - Y), np.log(1-AL).T))

    def update_parameters(self, parameters, grads):
        # Update the weiths and bieses of each layer
        parameters = params.copy()
        L = len(parameters) // 2
        for l in range(L):
            parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - self.learning_rate * grads["dW" + str(l+1)]
            parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - self.learning_rate * grads["db" + str(l+1)]
        return parameters

    def prediction(self, X):
        Z_1 = self.linear_activation_forward(X, self.W1, self.b1, "relu")
        A2, cache2 = self.linear_activation_forward(A1, self.W2, self.b2, "sigmoid")
        y_predict = A2
        # The unit step activation function returns only 1 or 0.
        return y_predict

      

In [ ]:
# Train neural network
#print("train_Y_Attractive shape: " + str(train_Y_attractive.shape))

my_nn = TwoLayerNN(learning_rate=0.01, num_ite=1000)
my_nn.fit(train_X, train_Y_attractive)

0
dz: (20, 2000)
W.T: (20, 1)


ValueError: ignored

In [ ]:
fit(train_X, train_Y_):

SyntaxError: ignored

In [8]:
# Implementing a 2 layer neural network

class DeepNN:

    def __init__(self, learning_rate=0.01, num_ite=1000):
        self.learning_rate = learning_rate
        self.num_ite = num_ite

    def initialize_parameters_deep(self, layer_dims):
        parameters = {}
        L = len(layer_dims) # number of layers in the network
        for l in range(1, L):
            parameters["W" + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
            parameters["b" + str(l)] = np.zeros((layer_dims[l], 1))
        return parameters

    def fit(self, X, Y, layers_dims):
        costs = []                              # to keep track of the cost
        # Initialize parameters
        parameters = self.initialize_parameters_deep(layers_dims)
        for i in range(0, self.num_ite):
            print(i)
            # Forward propagation: with a relu and then sigmoid activation function
            AL, caches = self.L_model_forward(X, parameters)
            print('A')
            # Cost function
            cost  = self.compute_cost(AL, Y)
            print('B')
            # Initializing backward propagation
            grads = self.L_model_backward(AL, Y, caches)
            print('C')
            # Update the paramters.
            parameters = self.update_parameters(parameters, grads, self.learning_rate)
            print('D')
            # Print the cost every 100 iterations
            if i % 100 == 0 or i == num_iterations - 1:
                print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
            if i % 100 == 0 or i == num_iterations:
                costs.append(cost)
        return parameters, costs

    def linear_forward(self, A, W, b):
        # Implement the linear part of a layer's forward propagation. ( CHANGE )
        Z = np.dot(W, A) + b
        cache = (A, W, b)  
        return Z, cache

    def L_model_forward(self, X, parameters):
        # Implement the linear part of a layer's forward propagation.
        caches = []
        A = X
        L = len(parameters) // 2 
        # loop every layer
        for l in range(1, L):
            A_prev = A 
            A, cache = self.linear_activation_forward(A_prev, parameters["W" + str(l)], parameters["b" + str(l)], "relu")
            caches.append(cache)
        AL, cache = self.linear_activation_forward(A, parameters["W" + str(L)], parameters["b" + str(L)], "sigmoid")
        caches.append(cache)
        return AL, caches

    def linear_backward(self, dZ, cache):
        # Implement the linear portion of backward propagation for a single layer ( CHANGE )
        A_prev, W, b = cache
        m = A_prev.shape[1]
        #print('m: ' + str(m))
        #print('linear_backward A')
        print('dZ: ' + str(dZ.shape))
        #print('dZ: ' + str(dZ))
        print('A_prev.T: ' + str(A_prev.T.shape))
        #print('A_prev.T: ' + str(A_prev.T))
        dW = (1/m) * np.dot(dZ, A_prev.T)
        db = (1/m) * np.sum(dZ, axis = 1, keepdims=True)
        print('YES')
        dA_prev = np.dot(W.T, dZ)
        print('YES, YES')
        
        return dA_prev, dW, db


    def L_model_backward(self, AL, Y, caches):
        grads = {}
        L = len(caches) # the number of layers
        m = AL.shape[1]
        Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
        dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
        current_cache = caches[L-1]
        print("B1")
        dA_prev_temp, dW_temp, db_temp = self.linear_activation_backward(dAL, current_cache, "sigmoid")
        print("B2")
        grads["dA" + str(L-1)] = dA_prev_temp
        grads["dW" + str(L)] = dW_temp
        grads["db" + str(L)] = db_temp
        for l in reversed(range(L-1)):
            print("B3")
            print('l = ' + str(l))
            current_cache = caches[l]
            dA_prev_temp, dW_temp, db_temp = self.linear_activation_backward(dA_prev_temp, current_cache, "relu")
            grads["dA" + str(l)] = dA_prev_temp
            grads["dW" + str(l + 1)] = dW_temp
            grads["db" + str(l + 1)] = db_temp
            print("b4")
        return grads

    def linear_activation_forward(self, A_prev, W, b, activation):
        # Implement the forward propagation for the LINEAR->ACTIVATION layer
        Z, linear_cache = self.linear_forward(A_prev, W, b)
        if activation == "sigmoid":
            A = 1/(1 + np.exp(Z))
        elif activation == "relu":
            A = np.maximum(Z, 0)
        cache = (linear_cache, Z)  # note th
        return A, cache

    def linear_activation_backward(self, dA, cache, activation):
        # Backward propagation.
        linear_cache, activation_cache = cache
        Z = activation_cache
        if activation == "relu":
          # Implement Back propagation witha relu activation function
            print('relu bitches')
            dsig = np.where(Z>0, 1, 0)
            print('dA: ' + str(dA.shape))
            print('dsig: ' + str(dsig.shape))
            dZ = dA * dsig
            print('dZ: ' + str(dZ.shape))
            dA_prev, dW, db = self.linear_backward(dZ, linear_cache)
            print('im out')
        elif activation == "sigmoid":
            print('sigmoid bitches')
          # Implement Back propagation witha sigmoid activation function
            dsig = 1/(1+np.exp(-Z)) * (1 - 1/(1+np.exp(-Z)))
            print('sigmoid dsig: ' + str(dsig.shape))
            print('sigmoid dA: ' + str(dA.shape))
            print('sigmoid dsig: ' + str(dsig.shape))
            dZ = dA * dsig    #= self.sigmoid_backward(dA, activation_cache)
            print('sigmoid dZ: ' + str(dZ.shape))
            dA_prev, dW, db = self.linear_backward(dZ, linear_cache)
        return dA_prev, dW, db

    def compute_cost(self, AL, Y):
        cost = -(1/m) * (np.dot(Y, np.log(AL).T) + np.dot((1 - Y), np.log(1-AL).T))

    def update_parameters(self, params, grads, learning_rate):
        # Update the weiths and bieses of each layer
        parameters = params.copy()
        L = len(parameters) // 2
        for l in range(L):
            parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
            parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        return parameters

    def prediction(self, X):
        Z_1 = self.linear_activation_forward(X, self.W1, self.b1, "relu")
        A2, cache2 = self.linear_activation_forward(A1, self.W2, self.b2, "sigmoid")
        y_predict = A2
        # The unit step activation function returns only 1 or 0.
        return y_predict

      

In [ ]:
layers_dims = [116412, 20, 7, 5, 1]
              
my_deepNN = DeepNN(learning_rate=0.01, num_ite=100)
parameters, costs = my_deepNN.fit(train_X, train_Y_attractive, layers_dims)
#parameters, costs = L_layer_model(train_x, train_y, layers_dims, num_iterations = 2500, print_cost = True)

0
A
B
B1
sigmoid bitches
sigmoid dsig: (1, 2000)
sigmoid dA: (1, 2000)
sigmoid dsig: (1, 2000)
sigmoid dZ: (1, 2000)
dZ: (1, 2000)
A_prev.T: (2000, 5)
YES
YES, YES
B2
B3
l = 2
relu bitches
dA: (5, 2000)
dsig: (5, 2000)
dZ: (5, 2000)
dZ: (5, 2000)
A_prev.T: (2000, 7)
YES
YES, YES
im out
b4
B3
l = 1
relu bitches
dA: (7, 2000)
dsig: (7, 2000)
dZ: (7, 2000)
dZ: (7, 2000)
A_prev.T: (2000, 20)
YES
YES, YES
im out
b4
B3
l = 0
relu bitches
dA: (20, 2000)
dsig: (20, 2000)
dZ: (20, 2000)
dZ: (20, 2000)
A_prev.T: (2000, 116412)
YES
YES, YES
im out
b4
C
